## Balls in P

In [ ]:
import time
time0=time.time()

P is the group generated by p and q with the relations $q^-1p^2q=p^-2$ and $p^-1q^2p=q^-2$.
This document generates the balls B(n) in P of words of length up to n.

In this document any element is represented as a string, and the identity element is simply the empty string.
o will represent the inverse of p and w the inverse of q.
The first two balls are the following:

In [ ]:
B=[] #Saving all the balls as elements in the list B
B.append([''])
B.append(['','p','q','w','o'])

The size of the ball B(n) is qiven by the sequence A007904 i OEIS. It is as defined by the following function.

In [ ]:
# From OEIS: a(n) = 3*a(n-1) - 2*a(n-2) - 2*a(n-3) + 3*a(n-4) - a(n-5)
def sequence(n):
    L=[1,5,17,41,83,147]

    for i in range(6,n+1):
        L.append(3*L[i-1] - 2*L[i-2] - 2*L[i-3] + 3*L[i-4] - L[i-5])
    return L
sequence(15)

We know from Gardam that we need to look at elements of length 5 to find a counterexample to the unit conjecture. And in order to work with the product of two such elements we need B(10). Therefore this document find B(n) up to n=10.

### Identifying identities

The strategy is to generate balls of increasing n. To do so we look at the word combinations and reduce by known relations. One way is to identify any occurences of the identity. The most simple occurences of identities are elements multiplied by its inverse.

In [ ]:
inverses=['qw','wq','po','op']

The relations on P gives other words of length 6, that map to the identity. Gathering everything on either one side or the other yields the 4 combinations

In [ ]:
relation_identities=['wppqpp','oowooq','oqqpqq','wwowwp']

But any cyclic permutation of these elements will be equal to the identity. Hence we must generate them all. To do this we will first write a function.

In [ ]:
def cyclic_permutations(w):
    '''Given a string the function returns a list of all cyclic permutations of the string
    '''
    word,L=w,[]

    for i in range(len(word)):
        L.append(word[i:]+word[:i])

    return L

In [ ]:
all_relation_identities=[]
for w in relation_identities:
    all_relation_identities=all_relation_identities+cyclic_permutations(w)

print(all_relation_identities)

### Identity-reducing functions

Now that we have 'all' identities indified we can write a function that will replace any occurrence of the words above with the empty string.

In [ ]:
def remove_identity(w):
    '''The function removes any instance of 28 identity words
    - the 4 words of length 2 and the 24 of length 6 given by the realtion on P.
    The function needs the globally defined lists all_relation_identities and inverses'''
    word=w

    #mapping inverses to the identity
    for i in inverses:
        word=word.replace(i,'')

    if len(word)>=6: #Possibly saving some time
        #Doing the same for all the identities from the relations
    
        for i in all_relation_identities:
            word=word.replace(i,'')
    return word

And also a function that keeps doing this until the word does not have a subword matching any of the 28 identities.

In [ ]:
def repeat_remove_identity(w):
    '''This function repeatedly runs remove_identity, untill it does not have any of the 28 identity words in it.
    It requires remove_identity function as well as the two lists of identities
    '''
    word=w

    #The word is reduced by the remove_identity function
    word=remove_identity(word)

    #If the word is unchanged by the function above it is simply returned
    if word==w:
        return word        

    #otherwise it calls this function again
    return repeat_remove_identity(word)

In [ ]:
repeat_remove_identity('poqwopoqwwqopopowqoqw')

### The inverse function

It will be convinient to have a function that returns the inverse of a given word, by simply using the group theoretic property that an inverse of a product is the inverses of the factors multiplied in opposite order.

In [ ]:
def inverse(w):
    '''This function returns the inverse element of w'''
    #Inversed order
    word=w[::-1]

    #Swapping all a and o's as well as p and b's (via dummies called e)
    word=word.replace('p','e')
    word=word.replace('o','p')
    word=word.replace('e','o')
    word=word.replace('w','e')
    word=word.replace('q','w')
    word=word.replace('e','q')

    return word

In [ ]:
inverse('pwoppq')

### Sister swaps

If we take any of the 6-long identity-words and pull half of the word to the other side of the equality sign we get 12 pairs (as each pair will all occur twice) of words of length 3 that are in fact the same word, in this code I call these pairs sisters. To reduce words we will swap sisters, to see if any identities occur along the way. But let's first make lists containing the sisters

In [ ]:
sisters=[] #all sisters
for i in range(len(all_relation_identities)):
    sisters.append(all_relation_identities[i][:3])

sistertwin=[] #all sisters with the twin in the same index as in the list above
for i in range(len(all_relation_identities)):
    sistertwin.append(inverse(all_relation_identities[i][3:]))

print(all_relation_identities)
print((sisters))
print((sistertwin))

We will sort these sets, so everything containing two p's or two q's will occur first in the list of sisters (and thus last in the list of sistertwin). We need this to be able to identify similar words later.

In [ ]:
sisters_special=sisters[:6]+sisters[12:18]+sisters[6:12]+sisters[18:]
print(sisters_special)

In [ ]:
def reduce(w):
    ''' This function reduces w by running the repeat_remove_identity and then
    swapping sisters following the order of the sisters_special list, reducing for identity-occurences along the way
    '''
    word=w

    word=repeat_remove_identity(word)
    
    for i in sisters_special:
        word=word.replace(i,sistertwin[sisters.index(i)])
        word=repeat_remove_identity(word)

    return word

To ease work we create a function that can apply a function as above to every element of a list that by default removes any multiple occurrences.

In [ ]:
def reduce_list(lst,fnct,*args,keep_multiples=False):
    ''' given a list and a reducing-function, the fuction returns the list of reduced elements
        The default removes multiples'''
    new_lst=lst
    
    for i in range(len(new_lst)):
        new_lst[i]=fnct(new_lst[i],*args)
   
    if not keep_multiples:
        new_lst=set(new_lst)
        new_lst=list(new_lst)
    
    return new_lst

### Balls with words of length up to 4

We can now generate the balls up to length 4. We will use the itertools.product to generate all products of words and then reduce via the reduce function above until the number of elements in the list matches the number given by the integer sequence A007904 i OEIS.

In [ ]:
import itertools

In [ ]:
B.append([''.join(word) for word in itertools.product(B[1], B[1])])

In [ ]:
B[2]=reduce_list(B[2],reduce)
print(B[2])
len(B[2])

In [ ]:
B.append([''.join(word) for word in itertools.product(B[2], B[1])])

In [ ]:
B[3]=reduce_list(B[3],reduce)
print(B[3])
len(B[3])

In [ ]:
B.append([''.join(word) for word in itertools.product(B[3], B[1])])

In [ ]:
B[4]=reduce_list(B[4],reduce)
print(B[4])
len(B[4])

We can see by comparing to the squence that we have fully reduced the lists.

In [ ]:
sequence(5)

### Random reducing function and B(5)

Sometimes, in larger words, possibilities to reduce for identities will go unnoticed by the computer because of the order that it swaps through the sisters. Therefore we create a reduce function, that swaps sisters at random. Notice that even though I have set a seed I cannot seem to replicate the same 'random' sequence. This creates veriation in the order as well as specific elements in the list. To adress this I import a fixed version of B later on in the script.
ANOTHER PROBLEM: this also alterates the elements chosen as representatives from the smaller balls! How can we fix this?

In [ ]:
import random
random.seed(5)

In [ ]:
def reduce_random(w,n):
    ''' This function reduces w by running the repeat_remove_identity and then
    randomly (via random.randint) swapping sisters n times, reducing for identity-occurences along the way.'''
    word=w
    word=repeat_remove_identity(word)

    for i in range(n):
        r=random.randint(0,23)
        word=word.replace(sisters[r],sistertwin[r])
        word=repeat_remove_identity(word)

    return word

Running the above function a handful of times yielded a strategy that seemed the most efficient when reducing a list of elements - alternating between the reduce and the reduce random function. The following code helps us to that.

In [ ]:
def reduce_list_alt(lst,n):
    ''' Reduces list of words by alternating use of reduce_list(lst,reduce) and reduce_list(lst,reduce_random,100) n times'''
    short_lst=reduce_list(lst,reduce)
    
    for i in range(n+1):
        short_lst=reduce_list(short_lst,reduce)
        short_lst=reduce_list(short_lst,reduce_random,100)     

    return short_lst

Using this we can now also find the words of length up to 5.

In [ ]:
B.append([''.join(word) for word in itertools.product(B[4], B[1])])

In [ ]:
B[5]=reduce_list_alt(B[5],3)
print(B[5])
len(B[5])

As the above utilises a random swapping method, we do not know how many times to run the reduce random (even with the seed). We are aiming for 147 elements, so lets repeat.

In [ ]:
B[5]=reduce_list_alt(B[5],1)
len(B[5])

We have now suceeded in finding B(5).

### Balls up to B(10)

Let's automate the process and find all balls up to B(10).

In [ ]:
time1=time.time()
print(time1-time0)
for n in range(6,11):
    B.append([''.join(word) for word in itertools.product(B[n-1], B[1])])
    B[n]=reduce_list_alt(B[n],18)
    print(len(B[n]))
time2=time.time()
print(time2-time0)

In [ ]:
sequence(10)

Checking against the sequence we se if we need to run something again (if it was a longer list we could write a function that did this, but here, this will do). Potentially the following two further reductions will be needed.

In [ ]:
B[9]=reduce_list_alt(B[9],5)
print(len(B[9]))

In [ ]:
B[10]=reduce_list_alt(B[10],5)
print(len(B[10]))

We see by comparing the sequence to the length of the lists, that we most likely (depending on the 'random' process) have succedded in finding and fully reducing all balls up to B(10). In order to fix the exact list that we work with (both order and representatives), we import a previously exported version.

In [ ]:
#with open('B.txt','w') as h:
    #h.write(str(B))

In [ ]:
import ast
f = open("B.txt")
B=ast.literal_eval(f.read())

### Making equals-list

A key element of identifying units is to know which words in a product are in fact the same element, and that therefore need to be added. Therefore we write a code that can identify any word to an element in a reduced list.

In [ ]:
def find_equal(w,lst,n=100,r=1):
    ''' This function reduces the word w recursively (via both reduce and reduce_random) until it is found in the lst. 
        If it has not found a solution after 1000 runs it returns 'problem' '''
    word=w
    word=reduce(word)
    word=reduce_random(word,n)
    
    if word in lst:
        return word
    
    if r>1001:
        return 'problem'
    
    return find_equal(word,lst,n,r+1)

Seeing if the function works as anticipated, it can find something already on the list

In [ ]:
print(B[10][300])
find_equal(B[10][300],B[10])

And testing it on a random product of two words in B(5)

In [ ]:
test=B[5][120]+B[5][144]
print(test)
print(find_equal(test,B[10]))

In [ ]:
def equal_products_list(products,ball):
    '''This function takes two lists as input - a list of products and a list of ball elements.
    It reduces all the words in the products list using find_equal function and generates a list of lists called equals. 
    equals[i] will be all the indicies refering to the products list where the word matches ball[i].
    It also returns a list of all the indicies of words that did not succeed in being reduced'''
    
    equals=[]
    reduced_products=[find_equal(word,ball) for word in products]
    
    for w in ball:
        equals.append([index for index, word in enumerate(reduced_products) if word==w])

    problems=[index for index,word in enumerate(reduced_products) if word=='problem']
        
    return equals,problems

In [ ]:
products=[''.join(word) for word in itertools.product(B[5], B[5])]
len(products)

In [ ]:
random.seed(5)
time1=time.time()
equals,problems=equal_products_list(products,B[10])
time.time()-time1

In [ ]:
problems #Note whether there are problems DOES depend on the seed/random run.

Seeing that there are problems, we define the following problem solver - that simply runs the problematic words once more through 'find equal'

In [ ]:
def fix_problems(problems,products,ball):
    '''This code reruns the elements that returned 'problem' another 1001 times. 
    If the word is identified it its index added to the relevent list in the global list equals'''
    shortlist=[products[index] for index in problems] #Finding the words that the indicies refer to
    reduced_shortlist=[find_equal(word,ball) for word in shortlist] #Running the funcion on all these words
    
    global equals

    for i in range(len(reduced_shortlist)):
        word=reduced_shortlist[i]
        if word!='problem': 
            equals[ball.index(word)].append(problems[i]) #If the word is not a problem it's index is added to the list of the matching word

    problems=[problems[index] for index,word in enumerate(reduced_shortlist) if word=='problem'] #Otherwise the index is added to the new problems list
    return problems

In [ ]:
fix_problems(problems,products,B[10]) #obs to run this load code below

Here we reorder the products, s.t. the placement numbering fits the SAT-code. Then we export a file.

In [ ]:
for i in equals: #Note for the numbering that we are adding 1 as SAT has 1-index in stead of 0
    for j in range(len(i)):
        i[j]=i[j]+147*2+1

In [ ]:
with open('P_info_5.txt','w') as g:
    g.write(str([len(B[5]),equals]))

What was just done for B(5) is repeated for the smaller balls. The information is actually contained in the lists above, but extracting the relevant information is quite difficult and it is therefore easiest to run the function again.

In [ ]:
productslist=[0]
time1=time.time()
equalslist,problemslist=[],[]
for i in range(1,5):
    productslist.append([''.join(word) for word in itertools.product(B[i], B[i])])
    equals_hold,problems_hold=equal_products_list(productslist[i],B[2*i])
    equalslist.append(equals_hold)
    problemslist.append(problems_hold)
time.time()-time1

In [ ]:
problemslist

The numbering is changed to match what is needed in SAT

In [ ]:
for n in range(len(equalslist)):
    for i in equalslist[n]: #Note for the numbering that we are adding 1 as SAT has 1-index in stead of 0
        for j in range(len(i)):
            i[j]=i[j]+sq[n+1]*2+1

Lastly, these lists are exported

In [ ]:
for n in range(len(equalslist)):
    with open(f'P_info_{n+1}.txt','w') as g:
        g.write(str([len(B[n+1]),equalslist[n]]))